<a href="https://colab.research.google.com/github/RenjieCui/Master_Thesis/blob/main/Newton_LQ_flocking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Newton Iterations on the whole forward-backward system

In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import scipy.sparse.linalg
from datetime import datetime

**Setup:** In this MFC example, we take:
  - drift: $$b(x, m, a) 	= Ax + \bar{A} m + Ba$$
  - running cost:
    $$f(x, m, a)	= \frac{1}{2} \left[ Q x^2 + \bar{Q} \left(x - S m \right)^2 + R a^2 \right]$$
  - terminal cost:
    $$g(x, m) 	= \frac{1}{2} \left[ Q_T x^2 + \bar{Q}_T \left(x - S_T m \right)^2 \right]$$

In [ ]:
# ================================================== #
# PARAMETERS
# ================================================== #
# Running cost
Q =     0
R =     1.0 
Rinv =  1.0/R 
S =     1.0
kappa = 1.0
Qbar =  kappa**2
# Terminal cost
QT =    0
ST =    0
QbarT = 0
# Dynamics
A =     0
Abar =  0#
B =     1.0
sigma = 1.0
a =     0.5*sigma**2

xbar0 = 1.0 # initial mean
sig0 = 0.2 # initial standard deviation
x2bar0 = sig0**2 + xbar0**2 # second moment (assuming Gaussian)

# Time
T =     10.0
Nt =    10000 # number of points
Dt =    T/(Nt-1)
t_grid = np.linspace(0,T,Nt,endpoint=True) # grid on [0,T]
# Algorithm parameters
epsilon = 1.e-10

In [ ]:
# ================================================== #
# USEFUL FUNCTIONS
# ================================================== #
def array_from_fct(f):
  np.array([f(t) for t in t_grid])

def fct_from_array(arr):
  return lambda t : arr[(np.abs(t_grid - t)).argmin()]

In [ ]:
# ODE FOR 2nd ORDER COEFFICIENT P
def rate_ODE_P(P_t, t):
  dP_dt = -( (P_t*A+A*P_t) - P_t**2 * B**2 * Rinv + Q + Qbar )
  return dP_dt

def rate_ODE_P_reverse(P_s, t): 
  # here s plays the role of T-t: need to apply functions control and X at point t = (T-s)
  s = T-t
  dP_dt = rate_ODE_P(P_s, t)
  return -dP_dt

def get_P_vec():
  PT = QT + QbarT
  # reverse time for solution, treat YT as initial condition, use -rate*dt
  s_grid = np.flip(T-t_grid) # interpreted as reversed time
  # ues ODE solver forward in time
  PT0 = odeint(rate_ODE_P_reverse, PT, s_grid)
  P0T = np.flip(PT0, axis=0) # return P from 0 to T
  return np.reshape(P0T, Nt)

In [ ]:
# ================================================== #
# NEWTON ITERATIONS ON THE WHOLE FORWARD-BACKWARD SYSTEM
# ================================================== #

def F_op(P, X):
    z = X[:Nt]
    r = X[Nt:]
    F = np.zeros(2*Nt)
    # Z PART
    F[0] = z[0] - xbar0
    for n in range(1,Nt):
        F[n] = (z[n]-z[n-1])/Dt - ( (A+Abar-B**2*Rinv*P[n-1])*z[n] - B**2*Rinv*r[n-1] ) 
    # R PART
    for n in range(0, Nt-1):
        F[Nt+n] = - (r[n+1]-r[n])/Dt  - ((A - P[n]*B**2*Rinv)*r[n]+(P[n]*Abar - Qbar*S)*z[n+1])
        #F[Nt+n] = - (r[n+1]-r[n])/Dt - ( (A+Abar - P[n]*B**2*Rinv)*r[n] + (2*P[n]*Abar - 2*Qbar*S + Qbar*S**2)*z[n+1] )
    #F[Nt+Nt-1] = r[Nt-1] - (- 2*QbarT*ST + QbarT*ST**2 ) * z[Nt-1]
    F[Nt+Nt-1] = r[Nt-1] - (- QbarT*ST) * z[Nt-1]
    return F

def DF_sparsemat(P):
  # DIAGONAL
    MD = np.zeros(2*Nt)
    row_start_MD = 0
    col_start_MD = 0
    MD[0] = 1
    for n in range(1,Nt):
        MD[n] = 1.0/Dt - (A+Abar-B**2*Rinv*P[n-1])
    for n in range(0,Nt-1):
        MD[Nt+n] = 1.0/Dt - (A-P[n]*B**2*Rinv)
        #MD[Nt+n] = 1.0/Dt - (A+Abar-P[n]*B**2*Rinv)
    MD[2*Nt-1] = 1
    #-----------------------------------------------------------
    # BELOW diagonal: Lower
    ML = np.zeros(2*Nt)
    row_start_ML = 1
    col_start_ML = 0
    for n in range(0,Nt-1):
        ML[n] = -1.0/Dt
    # ABOVE diagonal: Upper
    MU = np.zeros(2*Nt)
    row_start_MU = 0
    col_start_MU = 1
    for n in range(Nt,2*Nt-1):
        MU[n] = -1.0/Dt
    #----------------------------------------------------------
    # DERIVATIVE OF F_z WRT r
    M_DrFz = np.zeros(2*Nt)
    row_start_M_DrFz = 1
    col_start_M_DrFz = Nt
    for n in range(0,Nt-1): #Nt-1 original code
        M_DrFz[n] = B**2 * Rinv
    #---------------------------------------------------------
    # DERIVATIVE OF F_r WRT z
    M_DzFr = np.zeros(2*Nt)
    row_start_M_DzFr = Nt
    col_start_M_DzFr = 1
    for n in range(0,Nt-1): #Nt-1 original code
        M_DzFr[n] = - (P[n]*Abar - Qbar*S)
        #M_DzFr[n] = - (2*P[n]*Abar - 2*Qbar*S + Qbar*S**2)
    #---------------------------------------------------------
    # DERIVATIVE OF F_r(terminal condition) WRT z
    M_DzFr_term = np.zeros(2*Nt)
    row_start_M_DzFr_term = 2*Nt-1
    col_start_M_DzFr_term = Nt-1
    M_DzFr_term[0] = -(- QbarT*ST )
    #M_DzFr_term[0] = -(- 2*QbarT*ST + QbarT*ST**2 )
    #==========================================================
    # CONSTRUCT the matrix
    # for spdiags we need to shift the vectors
    MD = np.roll(MD,col_start_MD)
    MU = np.roll(MU,col_start_MU)
    ML = np.roll(ML,col_start_ML)
    M_DrFz = np.roll(M_DrFz,col_start_M_DrFz)
    M_DzFr = np.roll(M_DzFr,col_start_M_DzFr)
    M_DzFr_term = np.roll(M_DzFr_term,col_start_M_DzFr_term)
    MDn = sparse.spdiags([MD, ML, MU, M_DrFz, M_DzFr, M_DzFr_term],\
                       [col_start_MD-row_start_MD, col_start_ML-row_start_ML, col_start_MU-row_start_MU, col_start_M_DrFz-row_start_M_DrFz, col_start_M_DzFr-row_start_M_DzFr, col_start_M_DzFr_term-row_start_M_DzFr_term],\
                       2*Nt, 2*Nt) # SPARSE matrix
    MO = MDn.tocsr() # matrix of the operator (lhs)
    return MO

def step_Newton(P, X_old):
  b = -F_op(P, X_old)
  MO = DF_sparsemat(P)
  X_tmp0 = np.asarray(np.mat( sparse.linalg.spsolve( MO, b ) ))
  X_tmp = X_tmp0[0]
  return X_tmp + X_old

def solver_Newton_zr(P, tol=1.e-8):
  z_old = np.zeros(Nt)
  r_old = np.zeros(Nt)
  X_old = np.concatenate((z_old, r_old))
  i = 1
  diff_z = []
  diff_r = []
  printstep = 1
  while(True):
    X_new = step_Newton(P, X_old)
    z_new = X_new[:Nt]
    r_new = X_new[Nt:]
    diff_z.append(np.linalg.norm(z_old-z_new)*np.sqrt(Dt))
    diff_r.append(np.linalg.norm(r_old-r_new)*np.sqrt(Dt))
    if (i % printstep == 0):
      print("\nNewton : iter = ", i)
      fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 3))
      iter_grid = np.linspace(1, len(diff_z), len(diff_z))
      print("diff_z = {}, \tdiff_r = {}".format(diff_z[-1], diff_r[-1]))
      # Plots
      axes[0].plot(iter_grid, diff_z, label="diff z")
      axes[0].plot(iter_grid, diff_r, label="diff r")
      axes[0].semilogy()
      axes[0].legend()
      axes[0].set_xlabel("iteration")
      axes[1].plot(t_grid, z_new, label="$z$")
      axes[1].plot(t_grid, r_new, label="$r$")
      axes[1].legend()
      axes[1].set_xlabel("time")
      fig.tight_layout()
      plt.show()
      if( diff_z[-1] < tol and diff_r[-1] < tol):
        break
    X_old = X_new.copy()
    z_old = X_old[:Nt]
    r_old = X_old[Nt:]
    i += 1
  return z_new, r_new, diff_z, diff_r


In [ ]:
# 0th ORDER COEFFICIENT s
def get_s_vec(z_fct, P_fct, r_fct):
    
    arrt = np.zeros(Nt)
    s = np.zeros(Nt)
    arrt[Nt-1] = 0.5 * z_fct(Dt*Nt)**2 * ST**2 * QbarT
    s[Nt-1] = arrt[Nt-1]
    #for it in range(Nt-1, -1, -1):
        #t = Dt*it
        #arrt[it] = a*P_fct(t) - 0.5*r_fct(t)**2*B**2*Rinv + r_fct(t)*Abar*z_fct(t) \
        #            + 0.5*z_fct(t)*S**2 * Qbar * z_fct(t)
        #arrt[it] *= Dt
        #s = np.cumsum(arrt[it:])
    for it in range(Nt-2, -1, -1):
        t = Dt*it
        arrt[it] = a*P_fct(t) - 0.5*r_fct(t)**2*B**2*Rinv + r_fct(t)*Abar*z_fct(t) \
                    + 0.5*z_fct(t)*S**2 * Qbar * z_fct(t)
        arrt[it] *= Dt
        s[it] = arrt[it] + s[it+1]
    return s

In [ ]:
def solver_Newton(tol=1.e-8):
  P = get_P_vec()
  z, r, diff_z, diff_r = solver_Newton_zr(P, tol)
  s = get_s_vec(fct_from_array(z), fct_from_array(P), fct_from_array(r))
  return z, P, r, s, diff_z, diff_r

In [ ]:
z, P, r, s, diff_z, diff_r = solver_Newton(tol=1.e-8)

In [ ]:
plt.plot(t_grid, z, label="$z$")
plt.plot(t_grid, P, label="$P$")
plt.plot(t_grid, r, label="$r$")
plt.plot(t_grid, s, label="$s$")
plt.legend()
plt.xlabel("time")
now = datetime.now()
filename = f"Newton-ODE-barx0={xbar0}-sig0={sig0}"+now.strftime("%Y-%m-%d-%H:%M:%S")+".png"
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
iter_grid = np.linspace(1, len(diff_z), len(diff_z))
plt.plot(iter_grid, diff_z, label="diff z")
plt.plot(iter_grid, diff_r, label="diff r")
plt.semilogy()
plt.legend()
plt.xlabel("iteration")
now = datetime.now()
filename = f"Newton-difference-barx0={xbar0}-sig0={sig0}"+now.strftime("%Y-%m-%d-%H:%M:%S")+".png"
plt.savefig(filename, dpi=300)
plt.show()

In [ ]:
def get_cost(z, P, r, s):
  u0m0 = 0.5*P[0]*x2bar0 + r[0]*xbar0 + s[0]
  # correction terms for MFC: u is not the value function!
  corr_T = (1-ST)*QbarT*ST*z[Nt-1]**2
  corr_int = - np.sum( (P*z + r)*Abar*z - (1-S)*Qbar*S*z**2 )*Dt
  return u0m0 + corr_T + corr_int
# Evaluation of the total cost
print("total cost = ", get_cost(z, P, r, s))
print("P[0] = {}, \t r[0] = {}, \t s[0] = {}".format(P[0], r[0], s[0]))

In [ ]:
from numpy import asarray
from numpy import savetxt

data = asarray([get_cost(z, P, r, s), P[0], r[0], s[0]])
filename = f"Newton-difference-barx0={xbar0}-sig0={sig0}.csv"
savetxt(filename, data, delimiter=',')